In [1]:
from pynq import Overlay
overlay = Overlay("./bit/forw_back.bit")  # ./bit/forw_back.bit  和  ./bit/model.bit
core = overlay.forw_back_0                # 和Vivado中自定义IP名字相同  overlay?可查

In [2]:
def net(flag, in_d, conv1, conv2, conv3, fc1, fc2, fc3, out, label):   # 向PL端口输入数据
    core.write(0x10, flag)    # 地址可在HLS下solution1/impl/misc/drivers/forw_back_v0_0/src/xforw_back_hw.h可查
    core.write(0x18, in_d)    # 地址也可在SDK下system.hdf文件IP blocks present in the design部分Registers可查
    core.write(0x20, conv1)
    core.write(0x28, conv2)
    core.write(0x30, conv3)
    core.write(0x38, fc1)
    core.write(0x40, fc2)
    core.write(0x48, fc3)
    core.write(0x50, out)
    core.write(0x58, label)
    core.write(0x00, 0x01)
    while(core.read(0x00)!=4):  # 如果0x00!=4就代表PL端还在运行
        i = 1
    return 0

In [3]:
def max10(data):          # 取最大值函数
    for i in range(10):
        if(data[i] == max(data)):
            location = i
            break
    return location

In [4]:
from pynq import Xlnk
import numpy as np

xlnk = Xlnk()            # 该接口必须申请内存后才能被IP使用，可以使用xlnk来申请一段连续内存缓冲区，该缓冲区允许PS跟PL之间进行有效的数据传输

In [5]:
# 给每个接口分配空间
data_in = xlnk.cma_array(shape=(30*30,), dtype=np.float32)
conv1 = xlnk.cma_array(shape=(9,), dtype=np.float32)
conv2 = xlnk.cma_array(shape=(9,), dtype=np.float32)
conv3 = xlnk.cma_array(shape=(9,), dtype=np.float32)
fc1 = xlnk.cma_array(shape=(576*180,), dtype=np.float32)
fc2 = xlnk.cma_array(shape=(180*45,), dtype=np.float32)
fc3 = xlnk.cma_array(shape=(45*10,), dtype=np.float32)
data_out = xlnk.cma_array(shape=(10,), dtype=np.float32)

In [6]:
from struct import unpack                  # 30*30图片数据结构：62无用字节 + 30行 * 每行4字节（四个字节有两个比特是多余的）

img = open("./Training_set/7/2.bmp", 'rb')  # 打开一张图片
img.seek(62)                                # 跳过前62个没用的字节
nums = []
and_list = [1, 2, 4, 8, 16, 32, 64, 128]    # 辅助从字节中提取比特
for i in range(120):
    num = unpack("B", img.read(1))[0]       # 从头开始逐一读取120个字节
    for j in range(8):                     # 遍历8个bit
        if (i % 4 == 3) and (j >= 6):      # 检测到两个多余bit就跳过
            continue
        nums.append(((num & and_list[7 - j]) == and_list[7 - j]))  # 真实数据就放到列表后面

x = np.array(nums, dtype=int)               # 把图片数据列表转换成数组

In [7]:
# 打开网络参数文件
para = open("./Network_parameter.bin", "rb")

# 读取卷积核参数
for i in range(9):
    data = para.read(4)
    conv1[i] = unpack("f", data)[0]

for i in range(9):
    data = para.read(4)
    conv2[i] = unpack("f", data)[0]

for i in range(9):
    data = para.read(4)
    conv3[i] = unpack("f", data)[0]
    
# 读取全连接系数矩阵参数
for i in range(576*180):
    data = para.read(4)
    fc1[i] = unpack("f", data)[0]

for i in range(180*45):
    data = para.read(4)
    fc2[i] = unpack("f", data)[0]

for i in range(45*10):
    data = para.read(4)
    fc3[i] = unpack("f", data)[0]

In [8]:
# 复制图片数据
for i in range(30*30):
    data_in[i] = x[i]
    # print(data_in[i], end=" " if i % 30 != 29 else "\n")

In [9]:
# 将图片数据传到网络计算
net(0, data_in.physical_address, conv1.physical_address, conv2.physical_address, conv3.physical_address, 
   fc1.physical_address, fc2.physical_address, fc3.physical_address, data_out.physical_address, 0)

0

In [10]:
# 输出识别结果（可能性最大的那一张）
print(max10(data_out))

7


In [11]:
# 输出实际结果
lbb = []
for i in range(10):
    lbb.append(i)

print(lbb[7])

7
